In [1]:
# %pip install fake-useragent
# %pip install webdriver-manager
# %pip install selenium
# %pip install undetected-chromedriver
# %pip install undetected-chromedriver -U
# %pip install free-proxy
# %pip install tqdm
# %pip install --upgrade fake-useragent

In [23]:
import os, random, time, json, itertools
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from fake_useragent import UserAgent
from tqdm import tqdm

In [129]:
class Viewbot:
    def __init__(self):
        self.config = json.load(open('data/config.json', 'r+'))
        self.ua = UserAgent()


    def open_url(self, mail, pass_, ua):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument("--incognito")
        self.options.add_argument('--start-maximized')
        self.options.add_argument('user-agent=%s' % ua.random)
        # self.options.add_argument("--proxy-server=%s" % proxy)
        self.options.headless = False
        data = []
        self.browser = uc.Chrome(options=self.options)
        lst = self.config['url']
        try:
            self.browser.get(r'https://www.upwork.com/freelance-jobs/')
            WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.nav-logo")))
        except:
            print('page not found')
            self.browser.quit()

        
        for keyword in lst:            
            searchbar = self.browser.find_element(By.XPATH, '//input[@type="search"]')  
            searchbar.send_keys(Keys.CONTROL, 'a')  
            searchbar.send_keys(keyword, Keys.RETURN)
            time.sleep(5)
            profile_dev = self.browser.find_element(By.XPATH, '//section[@data-ev-label="search_results"]')
            profiles_lst = profile_dev.find_elements(By.CSS_SELECTOR, "section.flex-1.section")
            prof_list = []
            for ele in profiles_lst:
                profile = ele.find_element(By.CSS_SELECTOR, "h4.title.m-0")
                prof_list.append(profile.find_element(By.TAG_NAME, "a").get_attribute("href"))
            data.append({"keyword":keyword, "profiles":prof_list[:5]})
        
        data_lst = [] 
        for profs in data:
            keyword = profs['keyword']
            prof_lst = profs['profiles']
            profiles = []
            for prof in prof_lst:
                try:
                    # Open a new tab
                    self.browser.execute_script("window.open('');")
                    # Switch to the new tab
                    self.browser.switch_to.window(self.browser.window_handles[-1])
                    # Give some time to let the new tab open
                    time.sleep(2)
                    # Navigate to the URL
                    self.browser.get(prof)
                    # Perform any actions needed on the new tab
                    time.sleep(5)  # Replace with actual actions
                    profile_name = self.browser.find_element(By.XPATH, '//h2[@itemprop="name"]').text
                    profile_loc = self.browser.find_element(By.CSS_SELECTOR, 'div.d-block.d-md-inline-block.location').text
                    try:
                        profile_badge = self.browser.find_element(By.CSS_SELECTOR, 'div.vetted-rated-badges.air3-popper-trigger').text
                    except:
                        profile_badge = ""
                    profile_headline = self.browser.find_element(By.CSS_SELECTOR, 'h2.mb-0.pt-lg-2x.h4').text
                    profile_desc = self.browser.find_element(By.CSS_SELECTOR, 'div.text-body.text-light-on-inverse.air3-line-clamp-wrapper').text
                    profile_skills = self.browser.find_element(By.CSS_SELECTOR, 'ul.d-flex.list-unstyled.flex-wrap-wrap.mb-0.air3-token-wrap').text
                    profiles.append({"Name":profile_name, "Location":profile_loc, "Badge":profile_badge, "Headline":profile_headline, 
                                     "Description":profile_desc, "Skills":profile_skills})

                except:
                    pass
            data_lst.append({"keyword":keyword, "profiles":profiles})
            time.sleep(2)

        self.browser.switch_to.window(self.browser.window_handles[0])
        
        with open('data.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)    
        self.browser.quit()

    def main(self):     
        self.open_url("", "", self.ua)
                
            

In [ ]:
bot = Viewbot()
bot.main()